# Feature Selection 
This file performs feature selection for SVC, logistic regression, AdaBoost, and the Bayes Classifier to PA classification. The feature selection is done by first creating a power set using all possible features, then training the models on the combinations of features and then storing the scores. This notebook is a helper notebook for CMM-model-assessment.ipynb.

## Load and Clean Data
First we import the necessary packages.

In [60]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

Next, we load and clean the data. Then we merge the 3 dataframes into one and then split the resulting dataframe into two. 
In the first group, we have the data corresponding to pharmacy fills in which a PA was not requested and in the second we have the data corresponding to prescriptions for which a PA was requested.  

In [61]:
# Load Data
df_date=pd.read_csv("data/dim_date.csv")
df_claim=pd.read_csv("data/dim_claims.csv")
df_pa=pd.read_csv("data/dim_pa.csv")
df_bridge=pd.read_csv("data/bridge.csv")

# Clean data so all reject_code values are integers
df_claim['reject_code'] = df_claim.reject_code.fillna(0).astype(int)

# Merge the data frames
df_main = pd.merge(df_claim, df_bridge, on='dim_claim_id')
df_main = pd.merge(df_main, df_pa, how='left', on='dim_pa_id')
df_main = pd.merge(df_main, df_date, how='left', on='dim_date_id')

# split the data frames into two -- PA requested or not
df_main_wPA = df_main[~np.isnan(df_main.pa_approved)].copy()
df_main_noPA = df_main[np.isnan(df_main.pa_approved)].copy()

Here is the dataframe corresponding to pharmacy fills with no PA requested. 

In [62]:
df_main_noPA.head()

,dim_claim_id,bin,drug,reject_code,pharmacy_claim_approved,dim_pa_id,dim_date_id,correct_diagnosis,tried_and_failed,contraindication,pa_approved,date_val,calendar_year,calendar_month,calendar_day,day_of_week,is_weekday,is_workday,is_holiday
1,2,999001,A,0,1,NaN,1,NaN,NaN,NaN,NaN,2017-01-01,2017,1,1,1,0,0,1
3,4,999001,A,0,1,NaN,1,NaN,NaN,NaN,NaN,2017-01-01,2017,1,1,1,0,0,1
4,5,417740,A,0,1,NaN,1,NaN,NaN,NaN,NaN,2017-01-01,2017,1,1,1,0,0,1
5,6,417740,A,0,1,NaN,1,NaN,NaN,NaN,NaN,2017-01-01,2017,1,1,1,0,0,1
6,7,999001,A,0,1,NaN,1,NaN,NaN,NaN,NaN,2017-01-01,2017,1,1,1,0,0,1


Here is the data frame corresponding to pharmacy fills with a PA requested.

In [63]:
df_main_wPA.head()

,dim_claim_id,bin,drug,reject_code,pharmacy_claim_approved,dim_pa_id,dim_date_id,correct_diagnosis,tried_and_failed,contraindication,pa_approved,date_val,calendar_year,calendar_month,calendar_day,day_of_week,is_weekday,is_workday,is_holiday
0,1,417380,A,75,0,1.0,1,1.0,1.0,0.0,1.0,2017-01-01,2017,1,1,1,0,0,1
2,3,417740,A,76,0,2.0,1,1.0,0.0,0.0,1.0,2017-01-01,2017,1,1,1,0,0,1
9,10,417380,A,75,0,3.0,1,0.0,0.0,1.0,1.0,2017-01-01,2017,1,1,1,0,0,1
10,11,999001,A,76,0,4.0,1,1.0,1.0,0.0,1.0,2017-01-01,2017,1,1,1,0,0,1
14,15,417614,A,70,0,5.0,1,0.0,1.0,0.0,1.0,2017-01-01,2017,1,1,1,0,0,1


Now we use one-hot encoding to turn the categorical features into binary features. There are two categorical features; the bin i.e. the payer and the drug type.

In [64]:
# encode all categorical features in the case that a PA was requested 
df_aug=df_main_wPA.copy()
df_aug['70'] = pd.get_dummies(df_aug['reject_code'])[70]
df_aug['75'] = pd.get_dummies(df_aug['reject_code'])[75]
df_aug['76'] = pd.get_dummies(df_aug['reject_code'])[76]
df_aug['bin417380']=pd.get_dummies(df_aug['bin'])[417380]
df_aug['bin999001']=pd.get_dummies(df_aug['bin'])[999001]
df_aug['bin417740']=pd.get_dummies(df_aug['bin'])[417740]
df_aug['bin417614']=pd.get_dummies(df_aug['bin'])[417614]
df_aug['drug_A']=pd.get_dummies(df_aug['drug'])['A']
df_aug['drug_B']=pd.get_dummies(df_aug['drug'])['B']
df_aug['drug_C']=pd.get_dummies(df_aug['drug'])['C']

Now we make the train-test split.

In [65]:
from sklearn.model_selection import train_test_split

# keep all except temporal features for now 
X=df_aug[['70', '75', '76', 'bin417380', 'bin999001','bin417740', 'bin417614','correct_diagnosis', 'contraindication', 'tried_and_failed', 'drug_A', 'drug_B','drug_C']]
y=df_aug[['pa_approved']]
X_train_gen,X_test_gen,y_train_gen,y_test_gen = train_test_split(X,y,
                                                test_size=.2,
                                                shuffle=True,
                                                stratify=y)

## Models of Interest
We will analyze the following classifiers: 
1. logistic regression;
2. Bayes classifier;
3. random forests;
4. linear support vector machine; 
5. AdaBoost. 

We first import the models and load the class for the Bayes/Group Average classifier.

In [66]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import LinearSVC
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import AdaBoostClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import VotingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.base import clone

In [67]:
from collections import Counter
from sklearn.base import BaseEstimator, TransformerMixin, ClassifierMixin

class GroupAverageClassifier(BaseEstimator, TransformerMixin, ClassifierMixin):
    
    def __init__(self):
        pass
    
    def fit(self, X, y):
        self.approved_count_ = Counter()
        self.total_count_ = Counter()
        for r, t in zip(X, y):
            g = tuple(r)
            self.approved_count_[g] += t
            self.total_count_[g] += 1
        return self
    
    def transform(self, X, y=None):
        return self.predict_proba(X)
    
    def predict(self, X):
        prob = self.predict_prob(X)
        pred = (prob > 0.5).astype(int)
        return pred
    
    def predict_prob(self, X):
        prob = np.zeros(X.shape[0])
        eps = 1e-4
        for i, r in enumerate(X):
            g = tuple(r)
            prob[i] = self.approved_count_[g] / (self.total_count_[g] + eps)
        return prob

## Metrics of Interest 
We will be interested in accuracy, recall, precision, f1 and the area under the roc curve. 

In [68]:
from sklearn.metrics import accuracy_score, precision_score, recall_score
from sklearn.metrics import f1_score, roc_auc_score, balanced_accuracy_score

In [69]:
def get_func_name(f):
    name = f.__name__
    if name.endswith('_score'):
        name = name[:-6]
    return name

default_scores = [accuracy_score, precision_score, recall_score,f1_score, roc_auc_score, balanced_accuracy_score]
default_names = [get_func_name(f) for f in default_scores]

We now write the functions that build the powerset from all of the features and evaluate the models on this collection of features. 

In [127]:
def powerset_no_empty(s):
    power_set = []
    x = len(s)
    for i in range(1 << x):
        power_set.append([s[j] for j in range(x) if (i & (1 << j))])
            
    return power_set[1:]

def get_all_scores(model):    
    cutoff = .5
    acc = np.empty(len(possible_features))
    precision = np.empty(len(possible_features))
    recall = np.empty(len(possible_features))
    f1 = np.empty(len(possible_features))
    roc = np.empty(len(possible_features))
    acc_t = np.empty(len(possible_features))
    precision_t = np.empty(len(possible_features))
    recall_t = np.empty(len(possible_features))
    f1_t = np.empty(len(possible_features))
    roc_t=np.empty(len(possible_features))

    df=pd.DataFrame(columns=['acc', 'prec', 'recall', 'f1', 'roc', 'acc_t','prec_t', 'recall_t', 'f1_t', 'roc_t' ])

    for j in range(len(possible_features)):
        ## get X and y
        print(j)
        X_mini = np.array(X_train_gen[possible_features[j]])

        y_train=np.array(y_train_gen).ravel()

        # Cloning the regression makes a fresh regression 
        # model for each run
        clone_reg = clone(model)

        # fit the model
        clone_reg.fit(X_mini, y_train)
        ## assign the value based on the cutoff
        y_train_pred = clone_reg.predict(X_mini)

        X_mini_t = np.array(X_test_gen[possible_features[j]])
        y_test=np.array(y_test_gen).ravel()
        
        y_test_pred = clone_reg.predict(X_mini_t)


        acc[j] = accuracy_score(y_train_pred,y_train)
        precision[j] = precision_score(y_train,y_train_pred)
        recall[j] = recall_score(y_train,y_train_pred)
        f1[j] = f1_score(y_train,y_train_pred)
        roc[j]=roc_auc_score(y_train, y_train_pred)

        acc_t[j] = accuracy_score(y_test_pred,y_test)
        precision_t[j] = precision_score(y_test,y_test_pred)
        recall_t[j] = recall_score(y_test,y_test_pred)
        f1_t[j] = f1_score(y_test,y_test_pred)
        roc_t[j]=roc_auc_score(y_test, y_test_pred)

    df['acc']=acc
    df['acc_t']=acc_t
    df['prec']=precision
    df['prec_t']=precision_t
    df['recall']=recall
    df['recall_t']=recall_t
    df['f1']=f1
    df['f1_t']=f1_t
    df['roc']=roc
    df['roc']=roc_t

    return df


The features we are interested in are given below. Recall that when we encode a categorical feature with $K$ classes via one-hot encoding, we only keet $K-1$ of the resulting binary features. 

In [128]:
possible_features=powerset_no_empty(['70', '75', 'bin417380','bin417740', 'bin417614','correct_diagnosis', 'contraindication', 'tried_and_failed', 'drug_A', 'drug_B'])

### Logistic Regression

In [71]:
log_reg=LogisticRegression()
df= get_all_scores(log_reg)

1023
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
2

In [94]:
df.sort_values(by='roc', ascending=False).head()

,acc,prec,recall,f1,roc,acc_t,prec_t,recall_t,f1_t,roc_t
0,0.735089,0.929749,0.691561,0.793159,0.773624,0.735302,0.929723,0.691896,0.793370,NaN
2,0.735089,0.929749,0.691561,0.793159,0.773624,0.735302,0.929723,0.691896,0.793370,NaN
656,0.784706,0.877648,0.821371,0.848578,0.754218,0.786134,0.878733,0.822284,0.849572,NaN
906,0.784706,0.877648,0.821371,0.848578,0.754218,0.786134,0.878733,0.822284,0.849572,NaN
916,0.784706,0.877648,0.821371,0.848578,0.754218,0.786134,0.878733,0.822284,0.849572,NaN


Our goal is to extract the features with the best roc score. However, the baseline model (see pa_classifier.ipynb) has an accuracy of .73, so we reject the first two feature combinations here. 

In [130]:
print('We will use features: ', possible_features[656], 'for logistic regression.')

We will use features:  ['70', 'bin417614', 'tried_and_failed', 'drug_B'] for logistic regression.


### Linear Support Vector Machine

In [78]:
SVC = LinearSVC(C=1)
df_SVC=get_all_scores(SVC)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

In [93]:
df_SVC.sort_values(by='roc', ascending=False).head()

,acc,prec,recall,f1,roc,acc_t,prec_t,recall_t,f1_t,roc_t
0,0.735089,0.929749,0.691561,0.793159,0.773624,0.735302,0.929723,0.691896,0.793370,NaN
2,0.735089,0.929749,0.691561,0.793159,0.773624,0.735302,0.929723,0.691896,0.793370,NaN
658,0.784706,0.877648,0.821371,0.848578,0.754218,0.786134,0.878733,0.822284,0.849572,NaN
176,0.784706,0.877648,0.821371,0.848578,0.754218,0.786134,0.878733,0.822284,0.849572,NaN
178,0.784706,0.877648,0.821371,0.848578,0.754218,0.786134,0.878733,0.822284,0.849572,NaN


Again, we reject the first two feature combinations because the accuracy is too low and we choose the next combination.

In [131]:
print('We will use features: ',possible_features[658],'for the linear support vector machine.')

We will use features:  ['70', '75', 'bin417614', 'tried_and_failed', 'drug_B'] for the linear support vector machine.


### AdaBoost

In [79]:
ada_clf = AdaBoostClassifier(DecisionTreeClassifier(max_depth=1),
            n_estimators = 10,
            algorithm="SAMME.R",
            learning_rate = 0.5
        )
df_ada=get_all_scores(ada_clf)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

In [125]:
df_ada.sort_values(by='roc', ascending=False).head()

,acc,prec,recall,f1,roc,acc_t,prec_t,recall_t,f1_t,roc_t
0,0.735089,0.929749,0.691561,0.793159,0.773624,0.735302,0.929723,0.691896,0.793370,NaN
2,0.735089,0.929749,0.691561,0.793159,0.773624,0.735302,0.929723,0.691896,0.793370,NaN
514,0.735089,0.929749,0.691561,0.793159,0.773624,0.735302,0.929723,0.691896,0.793370,NaN
838,0.807294,0.851357,0.893646,0.871989,0.732076,0.808285,0.851808,0.894605,0.872682,NaN
326,0.807294,0.851357,0.893646,0.871989,0.732076,0.808285,0.851808,0.894605,0.872682,NaN


This time, we reject the first three feature combinations for hacing accuracy scores that are too low. 

In [133]:
print('We will use features:',possible_features[838],'for AdaBoost.')

We will use features: ['70', '75', 'bin417380', 'contraindication', 'drug_A', 'drug_B'] for AdaBoost.


### Bayes Classifier

In [80]:
bayes_clf=GroupAverageClassifier()
df_bayes=get_all_scores(bayes_clf)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

In [87]:
df_bayes.sort_values(by='roc', ascending=False).head()

,acc,prec,recall,f1,roc,acc_t,prec_t,recall_t,f1_t,roc_t
0,0.735089,0.929749,0.691561,0.793159,0.773624,0.735302,0.929723,0.691896,0.793370,NaN
2,0.735089,0.929749,0.691561,0.793159,0.773624,0.735302,0.929723,0.691896,0.793370,NaN
669,0.784176,0.880888,0.816556,0.847503,0.757407,0.785720,0.881870,0.817790,0.848622,NaN
906,0.784706,0.877648,0.821371,0.848578,0.754218,0.786134,0.878733,0.822284,0.849572,NaN
904,0.784706,0.877648,0.821371,0.848578,0.754218,0.786134,0.878733,0.822284,0.849572,NaN


We reject the first two feature combinations and choose the third. 

In [134]:
print('We will use features: ', possible_features[669],'for the Bayes Classifier.')

We will use features:  ['75', 'bin417380', 'bin417740', 'bin417614', 'tried_and_failed', 'drug_B'] for the Bayes Classifier.
